## ひろしまQuest2020#stayhome【コース予測部門】

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [2]:
ALL_MERGE = 'intermediate/all_merge_17.f'
SUBMIT = 'submit/course_17.csv'

In [3]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 192)

In [4]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 192)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_straight,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_course00,bc_course01,bc_course02,bc_course03,bc_course04,bc_course05,bc_course06,bc_course07,bc_course08,bc_course09,bc_course10,bc_course11,bc_course12,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,...,salary_year_bat,salary_x_year_bat,bmi_bat,batter_cnt,bat_game_cnt,firm,height,weight,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.464734,0.110359,0.151086,0.097383,0.067492,0.043210,0.012933,0.052803,0.064837,0.095199,0.082009,0.040512,0.063252,0.053916,0.030534,0.046508,0.038328,0.163762,0.057728,0.130872,0.132543,0,0,0,0,0,1,0,0,0.000000,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.442351,0.093302,0.109348,0.078518,0.141711,0.065447,0.017669,0.051654,0.044893,0.071306,0.068422,0.032002,0.043180,0.036149,0.029388,0.042910,0.033625,0.152078,0.078157,0.203642,0.164248,0,0,0,0,0,1,0,0,0.200000,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
2,2,0.0,5.0,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.504212,0.069963,0.065568,0.046520,0.211722,0.059158,0.019231,0.023626,0.027656,0.045421,0.045238,0.025275,0.023810,0.027106,0.023077,0.032784,0.022711,0.202930,0.081319,0.285165,0.157509,0,0,0,0,0,1,0,0,0.383333,...,1875.0,120000.0,23.889463,523.0,119.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0
3,3,0.0,12.0,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.475961,0.101792,0.226048,0.066617,0.042131,0.018345,0.003343,0.065762,0.034319,0.043647,0.027090,0.054336,0.065374,0.045046,0.062653,0.097944,0.085507,0.098410,0.098527,0.055307,0.231840,0,0,0,0,0,1,0,0,1.016667,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1
4,4,0.0,8.0,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.463496,0.099763,0.194714,0.067542,0.091198,0.023901,0.004486,0.054898,0.024390,0.034832,0.025124,0.039481,0.047557,0.033526,0.049596,0.080512,0.069337,0.085243,0.121054,0.076923,0.312424,0,0,0,0,0,1,0,0,1.200000,...,320.0,154880.0,22.839506,268.0,81.0,0,178,83,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1


### train
- 行数: 257117

In [5]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 192)

### test
- 行数: 521650

In [6]:
test = all_pitch[all_pitch['course'].isnull()]
test.shape

(521650, 192)

In [7]:
del all_pitch
gc.collect()

60

In [8]:
train_d = train.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

In [9]:
test_d = test.drop([
    'No', 
    'course', 
    'ball'
], axis=1)

## Dataset作成

In [10]:
lgb_train = lgb.Dataset(train_d, train['course'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [11]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 13,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 8.769293390201968, 
        'lambda_l2': 3.913949617576324e-05, 
        'num_leaves': 6, 
        'feature_fraction': 0.4, 
        'bagging_fraction': 0.8391111798378441, 
        'bagging_freq': 4, 
        'min_child_samples': 50
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 2.32501 + 0.00074896
[200]	cv_agg's multi_logloss: 2.32094 + 0.000830694
[300]	cv_agg's multi_logloss: 2.32019 + 0.000866506
[400]	cv_agg's multi_logloss: 2.32011 + 0.000889116
[500]	cv_agg's multi_logloss: 2.32029 + 0.000964611
Best num_boost_round: 415
Best CV score: 2.3200530570975535
CPU times: user 59min 32s, sys: 1min 1s, total: 1h 34s
Wall time: 15min 10s


In [12]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [13]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 16min 38s, sys: 17 s, total: 16min 55s
Wall time: 4min 13s


### Feature Importance

In [14]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
113,course_4,338
110,course_1,341
30,bc_course05,341
112,course_3,349
54,ave_elapsed_batter,350
117,course_8,352
29,bc_course04,360
126,height_bat,362
116,course_7,367
119,course_10,367


In [15]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 2min 58s, sys: 512 ms, total: 2min 58s
Wall time: 47.1 s


## 結果出力

In [16]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 14)


,index,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,0.063517,0.138277,0.068359,0.036513,0.061349,0.029210,0.058063,0.056141,0.033883,0.177168,0.085662,0.101880,0.089978
1,1,0.052758,0.139885,0.065361,0.030556,0.055761,0.029858,0.045288,0.056797,0.040353,0.155059,0.087688,0.127366,0.113270
2,2,0.032032,0.089531,0.059616,0.023063,0.045540,0.024081,0.040946,0.052908,0.030029,0.163125,0.104733,0.200166,0.134230
3,3,0.046696,0.110933,0.058513,0.025971,0.049905,0.024011,0.047459,0.055534,0.038567,0.169061,0.104018,0.143260,0.126072
4,4,0.055083,0.129427,0.068248,0.036952,0.051780,0.031178,0.047883,0.065216,0.042023,0.150330,0.073771,0.118562,0.129545


In [17]:
submit.mean()

index    260824.500000
0             0.046511
1             0.068783
2             0.056287
3             0.046628
4             0.059893
5             0.047088
6             0.043072
7             0.063541
8             0.052998
9             0.110638
10            0.088098
11            0.138053
12            0.178409
dtype: float64

In [18]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [19]:
'signate submit --competition-id=276 ./{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=276 ./submit/course_17.csv --note feat=192_cv=2.3200530570975535'

### GCS

In [20]:
'gsutil cp ./{} gs://yasunobu_baseball/submit/'.format(SUBMIT)

'gsutil cp ./submit/course_17.csv gs://yasunobu_baseball/submit/'

### 評価結果
#### 17(192) 
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 415
- Best CV score: 2.3200530570975535
- Wall time: 15min 10s
- 評価結果  : 2.3607863

#### 16(181)  ...選手IDごとの2017年のコース実績
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 355
- Best CV score: 2.3203553166835866
- Wall time: 5min 8s
- 評価結果  : 2.3608998

#### 15(153)  ...左右で球種/コースを別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 932
- Best CV score: 2.3387434142884445
- Wall time: 9min 41s
- 評価結果  : 2.3552543

#### 14(153)  ...左右で球種を別に集計
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 909
- Best CV score: 2.340082741044867
- Wall time: 9min 40s
- 評価結果  : 2.3555548

#### 3(157)  ...前の投球からの時間差を追加
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 857
- Best CV score: 2.340703597440808
- Wall time: 9min 20s
- 評価結果  : 2.3554087

#### 2(156)  ...merge12のパラメータをチューニング
- 'learning_rate' : 0.1,
- 'num_leaves': 6,
- Best num_boost_round: 835
- Best CV score: 2.3419206656914
- Wall time: 9min 7s
- 評価結果  : 2.3555299

#### 2(156)  ...merge12(2017のコースデータを付加) 
- 'learning_rate' : 0.1,
- Best num_boost_round: 121
- Best CV score: 2.344104226715559
- Wall time: 12min 30s
- 評価結果  : 2.3564326

#### 1(143)  ...球種予測の11と同じデータ 
- 'learning_rate' : 0.1,
- Best num_boost_round: 122
- Best CV score: 2.3445089434874253
- Wall time: 12min 2s (local)
- 評価結果  : 2.3562648